In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from datetime import datetime

In [2]:
def melt(df, col_vals, key, value):
    # melt pandas dataframe
    # col_vars: list of the columns that will be melted
    # key: name of column needs to be generated
    # value: name of the column that contains the value of interest
    keep_vars = df.columns.difference(col_vals)
    melted = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted.append(melted_c)
    return pd.concat(melted)

In [3]:
# read the raw spreadsheet, downloanded from shared drive
c_recon = pd.ExcelFile("C:/celsius/Liquidity/freeze.xlsx")

stats = pd.read_excel(c_recon, "Coin Stats", header = None)
defi = pd.read_excel(c_recon, "DeFi Assets")
ftx = pd.read_excel(c_recon, "FTX Summary")
template = pd.read_excel("C:/celsius/Liquidity/coin_apy_template.xlsx", sheet_name = "APY", header = None)
template.columns = template.iloc[0]
template = template[template["Coin"].notnull()]
template

,Coin,Bank - Balances,Celsius Network,Celsius Network System,Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,Defi Benqi Deployment,Banker Joe LINK,Reward Desk,MATIC Staking 8,AAVE Avalanche,Sushi Staking,Deployment - 1INCH Staking (testing),FTX - Kairon2,DD-Elrond-EGLD,Deployment - 1INCH
0,Coin,Bank - Balances,Celsius Network,Celsius Network System,Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,Defi Benqi Deployment,Banker Joe LINK,Reward Desk,MATIC Staking 8,AAVE Avalanche,Sushi Staking,Deployment - 1INCH Staking (testing),FTX - Kairon2,DD-Elrond-EGLD,Deployment - 1INCH
1,Category,undeployed,undeployed,undeployed,undeployed,undeployed,CEL Treasury,CEL Users,Institutional Loans,Posted Collateral,...,Defi,Defi,Defi,Staking,Defi,defi,defi,CEL,NaN,defi
2,Tier,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,...,5,5,5,5,4,2,2,5,NaN,2
3,Default,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,wBTC (Y/N),N,N,N,N,N,N,N,N,N,...,Y,Y,Y,N,N,Y,Y,n,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,YFL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ZEC,NaN,0,0,0,0,0,NaN,0.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,ZRX,NaN,0,0,0,0,0,NaN,0.0557,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,ZUSD,NaN,0,0,0,0,0,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
coin_asset_liability = stats[[0,2,3]].copy()
coin_asset_liability.columns = coin_asset_liability.iloc[0]
coin_asset_liability.drop([0], inplace = True)
coin_asset_liability.dropna(axis = 0 , how = "all", inplace = True)
coin_asset_liability.reset_index(drop = True, inplace = True)
coin_asset_liability.at[0, "Coin/Asset"] = "Total"
coin_asset_liability = coin_asset_liability[coin_asset_liability["Coin/Asset"].notnull()]
coin_asset_liability.columns = ['Coin', 'Net Assets Total', 'Net Liabilities Total']
coin_asset_liability["Net Assets Total"] = coin_asset_liability["Net Assets Total"].astype("float")
coin_asset_liability["Net Liabilities Total"] = coin_asset_liability["Net Liabilities Total"].astype("float")
#coin_asset_liability.columns
coin_asset_liability

,Coin,Net Assets Total,Net Liabilities Total
0,Total,2.480003e+10,-2.454838e+10
1,1INCH,6.783936e+06,-6.634962e+05
2,3CRV,5.978268e+02,0.000000e+00
3,AAVE,2.072371e+05,-2.021328e+05
4,ADA,2.439389e+08,-2.377736e+08
...,...,...,...
88,YFI,6.389648e+01,-2.161571e+01
89,YFL,3.153030e+03,0.000000e+00
90,ZEC,1.214472e+05,-1.218905e+05
91,ZRX,1.500202e+07,-1.500291e+07


In [5]:
# get the coin price, adding stable coins and srm_locked
coin_price = stats[[0,1]]
coin_price.columns = ["Coin", "Price"]
coin_price.drop([0], inplace = True)
coin_price.dropna(axis = 0 , how = "all", inplace = True)
coin_price = coin_price[coin_price["Coin"].notnull()]
coin_price.reset_index(drop = True, inplace = True)
coin_price.loc[len(coin_price.index)] = ['Stable Coins', 1] 
srm_price = float(coin_price.loc[coin_price["Coin"] == "SRM", "Price"])
coin_price.loc[len(coin_price.index)] = ['SRM_LOCKED', srm_price] 
coin_price


C:\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-5-a6d6eecd119e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coin_price.dropna(axis = 0 , how = "all", inplace = True)


,Coin,Price
0,1INCH,3.748692
1,3CRV,1
2,AAVE,237.478841
3,ADA,1.582577
4,ALCX,325.68
...,...,...
89,ZEC,257.056065
90,ZRX,1.12189
91,ZUSD,1
92,Stable Coins,1


In [6]:
tiers = template[template["Coin"]=="Tier"].T
tiers.reset_index(inplace = True)
tiers.columns = ["Account", "Tier"]
tiers.drop([0], inplace = True)
# there may be duplicate records in tier info
tiers.drop_duplicates(inplace = True)
# there are some rare occasions that the same account is assinged to two or more different tiers, use the first encountered
# the others are likely for some testing purposes, which will usually added to the end 
tiers = tiers[~tiers.Account.duplicated()]
filter4 = tiers['Tier'].isnull()
tiers.at[filter4, "Tier"] = "unassigned"
tiers["Tier"] = tiers["Tier"].astype("string")
#print(len(tiers))
#print(len(tiers[tiers["Tier"] == "unassigned"]))
#print(tiers.Account.nunique())
tiers["Tier"] = tiers["Tier"].apply(lambda x:x.split(".")[0])
#tiers["Tier"].unique()
tiers

,Account,Tier
1,Bank - Balances,1
2,Celsius Network,1
3,Celsius Network System,1
4,Celsius Network Finance,1
5,Celsius OTC,1
...,...,...
123,Sushi Staking,2
124,Deployment - 1INCH Staking (testing),2
125,FTX - Kairon2,5
126,DD-Elrond-EGLD,unassigned


In [7]:
categories = template[template["Coin"].isin(["Coin", "Category"])].T
categories.reset_index(drop = True, inplace = True)
categories.columns = ["Account", "Category"]
categories.drop([0], inplace = True)
categories.Category.fillna(value = "unassigned", inplace = True)
#print(len(categories))
#print(len(categories[categories["Category"] == "unassigned"]))
#print(categories.Account.nunique())
categories

,Account,Category
1,Bank - Balances,undeployed
2,Celsius Network,undeployed
3,Celsius Network System,undeployed
4,Celsius Network Finance,undeployed
5,Celsius OTC,undeployed
...,...,...
123,Sushi Staking,defi
124,Deployment - 1INCH Staking (testing),defi
125,FTX - Kairon2,CEL
126,DD-Elrond-EGLD,unassigned


In [8]:
cofa_original = pd.read_excel("C:/celsius/Liquidity/coin_apy_template.xlsx", sheet_name = "COFA")
cofa_original.dropna(axis=0, how='all', inplace=True)
cofa_original.reset_index(drop = True, inplace = True)
#display(cofa_original)
cofa_original.at[cofa_original["Coin"] == "stable", "Coin"] = "Stable Coins"
cofa_melt_cols = list(cofa_original.columns)
cofa_melt_cols.remove("Coin")
#print(cofa_melt_cols)
cofa = melt(cofa_original, cofa_melt_cols, "Account", "COFA")
cofa = cofa[cofa["COFA"].notnull()]
cofa.reset_index(drop = True, inplace = True)
cofa


,Coin,Account,COFA
0,1INCH,Default,0.0391
1,AAVE,Default,0.0384
2,ADA,Default,0.0214
3,BAT,Default,0.0095
4,BCH,Default,0.0183
...,...,...,...
68,ETH,FTX - LONG1,0.1335
69,check,FTX - LONG1,2.0000
70,BTC,Celsius Borrows Account,0.0000
71,ETH,Celsius Borrows Account,0.0000


In [9]:
apy = template[~template["Coin"].isin(["Coin", "Category", "Tier"])]
#apy.fillna(value = 0, inplace = True)
apy.replace([" ", "", "  "], np.nan, inplace = True)
apy =apy[apy["Coin"] != "wBTC (Y/N)"]
apy

C:\anaconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Coin,Bank - Balances,Celsius Network,Celsius Network System,Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,Defi Benqi Deployment,Banker Joe LINK,Reward Desk,MATIC Staking 8,AAVE Avalanche,Sushi Staking,Deployment - 1INCH Staking (testing),FTX - Kairon2,DD-Elrond-EGLD,Deployment - 1INCH
3,Default,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
5,1INCH,NaN,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0.1008
6,3CRV,NaN,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AAVE,NaN,0,0,0,0,0,NaN,0.0659,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0094,NaN,NaN,NaN,NaN
8,ADA,NaN,0,0,0,0,0,NaN,0.0743,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,YFL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ZEC,NaN,0,0,0,0,0,NaN,0.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,ZRX,NaN,0,0,0,0,0,NaN,0.0557,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,ZUSD,NaN,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
apy2 = apy.copy()
apy2["Coin"] = apy2["Coin"] + "_APY"
apy5 = apy2.T
apy5.reset_index(inplace = True)
apy5.columns = apy5.iloc[0]
apy5.rename(columns = {"Coin": "Account"}, inplace = True)
apy5.drop([0], inplace = True)
apy5

,Account,Default_APY,1INCH_APY,3CRV_APY,AAVE_APY,ADA_APY,ALCX_APY,ALPHA_APY,alUSD_APY,AMPL_APY,...,WDGLD_APY,XAUT_APY,XLM_APY,XRP_APY,YFI_APY,YFL_APY,ZEC_APY,ZRX_APY,ZUSD_APY,check_APY
1,Bank - Balances,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Celsius Network,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,0,0,0,80
3,Celsius Network System,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,0,0,0,78
4,Celsius Network Finance,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,0,0,0,NaN
5,Celsius OTC,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,0,0,0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Sushi Staking,NaN,NaN,NaN,0.0094,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,Deployment - 1INCH Staking (testing),NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,FTX - Kairon2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,DD-Elrond-EGLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# conver dyptes to float
for col in apy5.columns:
    if col != "Account":
        apy5.loc[apy5[col].isin(template["Coin"].unique()), col] = np.nan
        apy5[col] = apy5[col].astype("float")

In [12]:
# get the user collateral and inst collateral data
collateral = stats.copy()
collateral_p1 = collateral.iloc[:, 0].to_frame()
collateral_p1.columns = ["Coin"]
collateral_p2 = collateral.iloc[:, 4:]
collateral_p2.columns = collateral_p2.iloc[2]
collateral = pd.concat([collateral_p1, collateral_p2[["User Collateral", "Inst Collateral"]]], axis = 1)
collateral.drop([0,1,2,3], inplace = True)
collateral = collateral[collateral["Coin"].notnull()]
collateral["User Collateral"] = -1 * collateral["User Collateral"]
collateral["Inst Collateral"] = -1 * collateral["Inst Collateral"]
collateral.fillna(value = 0, inplace = True)
collateral

,Coin,User Collateral,Inst Collateral
4,1INCH,1.140411e+04,0.00
5,3CRV,0.000000e+00,0.00
6,AAVE,2.267784e+04,16710.93
7,ADA,4.946831e+07,60661894.34
8,ALCX,0.000000e+00,0.00
...,...,...,...
91,YFI,0.000000e+00,0.00
92,YFL,0.000000e+00,0.00
93,ZEC,2.501434e+04,0.00
94,ZRX,6.433919e+05,0.00


In [13]:
# process stats table

# first fill the "asset" or "liability" into row 0
stats.iloc[0] = stats.iloc[0].ffill()
# drop the secondary description of asset or liability (no use)
stats.drop([1], inplace = True)

#split stats into two parts, p1 is coin name and summary, p2 is assets/liabilities
stats_p1 = stats.iloc[:, 0:3]
stats_p1.columns = stats_p1.iloc[0]
stats_p1.reset_index(drop = True, inplace = True)
stats_p1.drop([0, 1, 2], inplace = True)
stats_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)


stats_p2 = stats.iloc[:, 4:]

# filter according to "assets" and the detailed account name cannot be null
stats_p3 = stats_p2.loc[:, stats_p2.loc[2].notnull()]
stats_p4 = stats_p3.loc[:, stats_p3.loc[0] == "Assets"]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.columns = stats_p4.iloc[1]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.drop([0, 1, 2], inplace = True)
stats = pd.concat([stats_p1["Coin"], stats_p4], axis = 1)
stats = stats[stats["Coin"].notnull()]
stats

C:\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Coin,Bank - Balances,Celsius Network,Celsius Network System,Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,Stakehound,Kraken Staking,Direct Staking,Mining,EAM - Balances,Others - Asset,BITFINEX,COINBASEPRO,DERIBIT,LIQUID
3,1INCH,0,0,117269.1238,0,0,0,0,0,0,...,0,0,0,0,0,6666666.66,0,0,0,0
4,3CRV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,0,9356.881043,354.787382,4.32075,0,0,19620.65,0,...,0,0,0,0,0,0,0,0,0,0
6,ADA,0,3,5422957.36,14728.44519,0,0,0,65511559,0,...,0,0,172989645.6,0,0,0,0,0,0,0
7,ALCX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,YFI,0,0.029842,0.308855,0,0,0,0,50.86,0,...,0,0,0,0,0,0,0.901743,0,0,0
91,YFL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92,ZEC,0,166.3544,95650.3996,0,0,0,0,21000,0,...,0,0,0,0,0,0,4630.434849,0,0,0
93,ZRX,0,196830.148,1218730.668,0,272.88,0,0,3500000,0,...,0,0,0,0,0,0,577363.3009,8735.95141,0,0


In [14]:
defi_p1 = defi.iloc[:, 0:3]
defi_p1.drop([0, 1, 2], inplace = True)
defi_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)
defi_p1
defi_p2 = defi.iloc[:, 3:]

# filter according the detailed account name cannot be null
defi_p3 = defi_p2.loc[:, defi_p2.iloc[1].notnull()]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.columns = defi_p3.iloc[1]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.drop([0, 1, 2], inplace = True)
defi_p3
defi = pd.concat([defi_p1["Coin"], defi_p3], axis = 1)
defi = defi[defi["Coin"].notnull()]
defi

C:\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Coin,Deployment - 1INCH,YFL,Deployment - Curve,AAVE Deployment / Staking,Celsius Borrows Account,Deployment Team - Misc,Network Deposits,OmniMan1,OmniMan2,...,TEST-ARBITRUM,TEST-AVALANCHE,TEST-FANTOM,Truefi Borrows Account,YD - Curve - BBTC,YD - Curve - pBTC,YD - Vesper - vETH,YD - Vesper - vLINK,Yield Desk - Compound,DD-Elrond-EGLD
3,1INCH,0,0.179374,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3CRV,0,0,597.826821,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,0,0,173546.1487,3936.348045,122.770498,0.001296,10.075529,228.097904,...,0,0,0,0,0,0,0,0,0,0
6,ADA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ALCX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,YFI,0,0,0,0,0,0,0,0.425842,0,...,0,0,0,0,0,0,0,0,0,0
91,YFL,0,109,0,0,0,0,3044.03038,0,0,...,0,0,0,0,0,0,0,0,0,0
92,ZEC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,ZRX,0,0,0,0,3237483.922,200000.8951,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# process ftx summary data
ftx.dropna(axis = 0 , how = "all", inplace = True)
ftx.dropna(axis = 1 , how = "all", inplace = True)
filter_ftx = ((ftx["Total Asset"] == 0) & (ftx["Total Borrow"] == 0))
ftx = ftx[(ftx["coin"].notnull()) & ~filter_ftx]
ftx.drop(columns = ["Total Asset", "Total Borrow"], inplace = True)
ftx_cols = []
for col in ftx.columns:
    ftx_cols.append("FTX - " + col)
ftx.columns = ftx_cols
ftx.rename(columns = {"FTX - coin": "Coin"}, inplace = True)

# below lines of code will merge BTC and WBTC together
ftx_btc = ftx[ftx["Coin"].isin(["BTC", "WBTC"])]
ftx_btc = ftx_btc.append(ftx_btc.sum(numeric_only=True), ignore_index=True)
ftx_btc = ftx_btc[~ftx_btc["Coin"].isin(["BTC", "WBTC"])]
ftx_btc.reset_index(drop = True, inplace = True)
ftx_btc.loc[0, "Coin"] = "BTC"
ftx = pd.concat([ftx_btc, ftx[~ftx["Coin"].isin(["BTC", "WBTC"])]])
ftx

,Coin,FTX - CnC,FTX - TEAM Directional,FTX - Grayscale,FTX - Main Account,FTX - DeFi,FTX - Management,FTX - Kairon,FTX - Kairon2,FTX - Brad,FTX - Jacob,FTX - cel_staking,FTX - Directional Trading 2,FTX - CEL,FTX - Borrow
0,BTC,4.862692e+03,0.000000e+00,0.000000,0.000000e+00,7.000210e+00,1.330789e-02,4.888338,0.0,0.000000e+00,2.611784e+04,0.0,0.000000e+00,3.558694e+02,1.000005e+03
0,USD,3.291748e+07,-1.125132e+06,-17.847001,2.095910e-01,5.009077e+06,5.450376e+06,222508.441600,0.0,-5.117511e+05,0.000000e+00,0.0,1.345841e+05,8.412648e+06,-3.129191e+06
1,LRC,9.000212e+05,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2,SOL,2.500050e+04,0.000000e+00,0.000000,2.000000e-08,0.000000e+00,0.000000e+00,0.000000,0.0,1.010000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-1.003395e+00
3,FTM,5.000133e+05,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
4,ALICE,2.000053e+04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
5,MANA,5.000141e+05,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
11,BOBA,0.000000e+00,0.000000e+00,0.000000,9.014767e+04,0.000000e+00,9.362582e+04,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
12,POLIS,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,2.135589e+05,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
13,RAY,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,1.629927e+05,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00


In [16]:
# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
#print("Updated at:", dt_string)

In [17]:
# define the stable coins
stables = ['alUSD','BUSD', 'GUSD', 'LUSD', 'LUSD Curve','MCDAI', 'PAX', 'SUSD', 'TUSD', 
           'USDC', 'USDT ERC20', 'ZUSD', "USD"]


In [18]:
# get a pandas df containing all tiers for merging
tier_dict = {"Tier":["1", "2", "3", "4", "5", "unassigned"]}
coin_tiers = pd.DataFrame.from_dict(tier_dict)
coin_tiers

,Tier
0,1
1,2
2,3
3,4
4,5
5,unassigned


In [19]:
# merge all 3 parts (coin stats, defi assets and ftx summary) together 
coin_stats = stats.merge(defi, on = "Coin", how = "outer")
coin_stats = coin_stats.merge(ftx, on = "Coin", how = "outer")
coin_stats.fillna(value = 0, inplace = True)
coin_stats_col = []
# the follwoing code is to remove the spaces trailing or following the name of the columns
for col in coin_stats.columns:
    coin_stats_col.append(col.strip())
coin_stats.columns = coin_stats_col
#len(coin_stats.columns)
coin_stats

,Coin,Bank - Balances,Celsius Network,Celsius Network System,Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,FTX - DeFi,FTX - Management,FTX - Kairon,FTX - Kairon2,FTX - Brad,FTX - Jacob,FTX - cel_staking,FTX - Directional Trading 2,FTX - CEL,FTX - Borrow
0,1INCH,0.0,0.0,1.172691e+05,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,3CRV,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,AAVE,0.0,0.0,9.356881e+03,354.787382,4.32075,0.0,0.0,19620.65,0.0,...,0.000062,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,ADA,0.0,3.0,5.422957e+06,14728.445190,0.00000,0.0,0.0,65511559.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,ALCX,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,LRC,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
93,ALICE,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
94,POLIS,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,213558.860739,0.0,0.0,0.000000,0.000000,0.0
95,SRM_LOCKED,0.0,0.0,0.000000e+00,0.000000,0.00000,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.0,6784.779347,0.0,0.0,13595.335935,33658.637474,0.0


In [20]:
# get a list of coins 
coin_list = list(coin_stats["Coin"].unique())
coin_list.append("Stable Coins")
#coin_list

In [21]:
# get a list of accounts
acc_list = np.asarray(tiers["Account"])
len(acc_list)

127

In [22]:
# see which accounts in coin recon sheet are not in covered in apy/cofa sheet
not_covered = []
for col in coin_stats.columns:
    if col not in acc_list:
        not_covered.append(col)
not_covered    

['Coin',
 'DeFi Borrows - Assets',
 'DeFi Assets - Assets',
 'FTX',
 'Deribit - API',
 'YD - Convex - bBTC',
 'YD - Curve - SBTC',
 'YD - Curve - hBTC',
 'TEST-MATIC',
 'DD-SHIBASWAP-WBTC',
 'YD - Keeper - renBTC',
 'DD-CONVEX-ALETH',
 'YD - Curve - AnkrETH',
 'DD-NOTIONAL-ETH',
 'DD-NOTIONAL-WBTC',
 'TEST-ARBITRUM',
 'TEST-AVALANCHE',
 'TEST-FANTOM',
 'DD-Elrond-EGLD',
 'FTX - Directional Trading 2']

In [23]:
# see which accounts in apy/cofa sheet are not listed in coin recon sheet
not_covered = []
for col in acc_list:
    if col not in coin_stats.columns:
        not_covered.append(col)
not_covered 

['Deployment- 1INCH Staking (TESTING)',
 'Impermanent_loss_hedge__RonSabo',
 'Hedge_Options',
 'FTX ',
 'FTX Summery',
 'FTX - Johannes',
 2,
 'Binance - CnC',
 'Binance - Main',
 'Binance - Earn',
 'Binance - DeFi',
 'Binance - RS',
 'Binance Staking',
 'OKEX',
 'YD - Curve - renBTC',
 'Deployment - Stable Coin Swaps',
 'Deployment Team - COMP supply',
 'Deployment- 1INCH Staking',
 'Convex: cvxBUSD3CRV-f',
 'FTX - LONG1',
 'Deployment - 1INCH Staking (testing)',
 'DD-Elrond-EGLD ']

In [24]:
# transpose coin_stats so we can merge with Tier/APY/COFA on accounts
coin_stats_t = coin_stats.T
new_header = coin_stats_t.iloc[0] #grab the first row for the header
coin_stats_t = coin_stats_t[1:] #take the data less the header row
coin_stats_t.columns = new_header #set the header row as the df header
coin_stats_t.reset_index(inplace = True)
coin_stats_t.rename(columns={coin_stats_t.columns[0]: "Account" }, inplace = True)
coin_stats_t.reset_index(drop=True, inplace = True)
coin_stats_t = coin_stats_t[~coin_stats_t["Account"].isin(['DeFi Borrows - Assets', 'DeFi Assets - Assets', "FTX"])]

coin_stats_t


Coin,Account,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,ANKR,...,YFI,YFL,ZEC,ZRX,ZUSD,LRC,ALICE,POLIS,SRM_LOCKED,TRX
0,Bank - Balances,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Celsius Network,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.029842,0.0,166.3544,196830.148,0.0,0.0,0.0,0.0,0.0,0.0
2,Celsius Network System,117269.1238,0.0,9356.881043,5422957.36,0.0,0.0,0.0,204.061424,24004.58859,...,0.308855,0.0,95650.3996,1218730.668,536684.7994,0.0,0.0,0.0,0.0,0.0
3,Celsius OTC,0.0,0.0,354.787382,14728.44519,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Celsius Network Finance,0.0,0.0,4.32075,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,272.88,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,FTX - Jacob,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.370193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,FTX - cel_staking,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,FTX - Directional Trading 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13595.335935,0.0
122,FTX - CEL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33658.637474,0.0


In [25]:
# merge the tier info 
coin_stat_tier = tiers.merge(coin_stats_t, on = "Account", how = "right")
#coin_stat_tier["Tier"].fillna(value = "unknown", inplace = True)

# get the number of stable coins
coin_stat_tier['Stable Coins']= coin_stat_tier[stables].sum(axis=1)
coin_stat_tier.to_excel("C:/celsius/Liquidity/coin_stats_tier.xlsx", index = False)

#print(len(coin_stat_tier[coin_stat_tier["Tier"] == "unknown"]))
coin_stat_tier["Tier"] = coin_stat_tier["Tier"].astype("str")
coin_stat_tier

,Account,Tier,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,...,YFL,ZEC,ZRX,ZUSD,LRC,ALICE,POLIS,SRM_LOCKED,TRX,Stable Coins
0,Bank - Balances,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.926714e+07
1,Celsius Network,1,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,166.3544,196830.148,0.0,0.0,0.0,0.0,0.0,0.0,1.706039e+06
2,Celsius Network System,1,117269.1238,0.0,9356.881043,5422957.36,0.0,0.0,0.0,204.061424,...,0.0,95650.3996,1218730.668,536684.7994,0.0,0.0,0.0,0.0,0.0,7.644991e+07
3,Celsius OTC,1,0.0,0.0,354.787382,14728.44519,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.469141e+06
4,Celsius Network Finance,1,0.0,0.0,4.32075,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,272.88,0.0,0.0,0.0,0.0,0.0,0.0,3.102732e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,FTX - Jacob,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
117,FTX - cel_staking,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
118,FTX - Directional Trading 2,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13595.335935,0.0,1.345841e+05
119,FTX - CEL,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33658.637474,0.0,8.412648e+06


In [26]:
# creating a list for summing stable coin yields
stable_yield = []
for coin in stables:
    stable_yield.append(coin+"_yield")
stable_yield

['alUSD_yield',
 'BUSD_yield',
 'GUSD_yield',
 'LUSD_yield',
 'LUSD Curve_yield',
 'MCDAI_yield',
 'PAX_yield',
 'SUSD_yield',
 'TUSD_yield',
 'USDC_yield',
 'USDT ERC20_yield',
 'ZUSD_yield',
 'USD_yield']

In [27]:
# calculating the yield for each coin/account
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on = "Account", how = "left")
#coin_stat_tier_apy.fillna(value = 0, inplace = True)

#coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
for coin in coin_list:
    #print(coin)
    yield_name = coin + "_yield"
    coin_apy = coin + "_APY"
    if coin_apy in coin_stat_tier_apy.columns:
        coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
    else:
        coin_stat_tier_apy[yield_name] = 0
        coin_stat_tier_apy[coin_apy] = np.nan
coin_stat_tier_apy['Stable Coins_yield']= coin_stat_tier_apy[stable_yield].sum(axis=1)
coin_stat_tier_apy['Stable Coins_APY']= coin_stat_tier_apy['Stable Coins_yield']/ coin_stat_tier_apy['Stable Coins']
coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
coin_stat_tier_apy

In [28]:
# group by coin/tier
for coin in coin_list:
    #print(coin)
    coin_yield = coin+"_yield"
    coin_apy = coin+"_APY"
    df1 = coin_stat_tier_apy[[coin, coin_yield, "Tier"]]
    df2 = df1.groupby('Tier', as_index = False).agg({coin: "sum",
                                                    coin_yield: "sum"})
    df2[coin_apy] = df2[coin_yield]/ df2[coin]
    df2.drop(columns = [coin_yield], inplace = True)
    #display(df2)
    coin_tiers = coin_tiers.merge(df2, on = "Tier", how = "left")
coin_tiers.fillna(value = 0, inplace = True)
coin_tiers

,Tier,1INCH,1INCH_APY,3CRV,3CRV_APY,AAVE,AAVE_APY,ADA,ADA_APY,ALCX,...,ALICE,ALICE_APY,POLIS,POLIS_APY,SRM_LOCKED,SRM_LOCKED_APY,TRX,TRX_APY,Stable Coins,Stable Coins_APY
0,1,1.172691e+05,0.0,0.000000,0.0,9715.990470,0.000000,5.437689e+06,0.0000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,1.199384e+08,0.000000
1,2,0.000000e+00,0.0,597.826821,0.0,141.248221,0.011224,0.000000e+00,0.0000,0.0,...,0.000000,0.0,0.000000,0.0,50471.776129,0.000000e+00,0.000000,0.0,5.504567e+07,0.007449
2,3,1.793736e-01,0.0,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.0000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
3,4,0.000000e+00,0.0,0.000000,0.0,19659.193429,0.065771,6.551156e+07,0.0743,0.0,...,20000.527364,0.0,213558.860739,0.0,576735.842576,6.521445e-10,0.000003,0.0,1.267952e+09,0.103876
4,5,6.666667e+06,0.0,0.000000,0.0,177720.670178,0.062812,1.729896e+08,0.0450,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,1.461058e+09,0.111970
5,unassigned,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.0000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000


In [29]:
coin_tier_p1_cols = ["Tier"]
coin_tier_p2_cols = ["Tier"]
filter2 = coin_tiers["Tier"] != "unassigned"
for col in coin_tiers.columns:
    if col != "Tier" and "_APY" in col:
        coin_tier_p2_cols.append(col)
    elif col != "Tier":
        coin_tier_p1_cols.append(col)
    else:
        pass
print(coin_tier_p1_cols)
print(coin_tier_p2_cols)

['Tier', '1INCH', '3CRV', 'AAVE', 'ADA', 'ALCX', 'ALPHA', 'alUSD', 'AMPL', 'ANKR', 'ATLAS', 'AVAX', 'BADGER', 'BAL', 'BAT', 'BCH', 'BNB', 'BNT', 'BOBA', 'BOND', 'BOR', 'BSV', 'BTC', 'BTG', 'BUSD', 'CEL', 'COMP', 'CREAM', 'CRV', 'CVX', 'DASH', 'DIGG', 'DOT', 'EOS', 'ETC', 'ETH', 'FARM', 'FIS', 'FTM', 'FTT', 'GUSD', 'KNC', 'LDO', 'LINK', 'LPT', 'LQTY', 'LTC', 'LUNA', 'LUSD', 'LUSD Curve', 'MANA', 'MATIC', 'MCDAI', 'MKR', 'OMG', 'ONX', 'ORBS', 'PAX', 'PAXG', 'PNT', 'QI', 'RAY', 'REN', 'ROOK', 'SGA', 'SGR', 'SNX', 'SOL', 'SPARK', 'SRM', 'SUSD', 'SUSHI', 'TAUD', 'TCAD', 'TGBP', 'THKD', 'TRU', 'TUSD', 'UMA', 'UNI', 'USD', 'USDC', 'USDT ERC20', 'VSP', 'WDGLD', 'XAUT', 'XLM', 'XRP', 'YFI', 'YFL', 'ZEC', 'ZRX', 'ZUSD', 'LRC', 'ALICE', 'POLIS', 'SRM_LOCKED', 'TRX', 'Stable Coins']
['Tier', '1INCH_APY', '3CRV_APY', 'AAVE_APY', 'ADA_APY', 'ALCX_APY', 'ALPHA_APY', 'alUSD_APY', 'AMPL_APY', 'ANKR_APY', 'ATLAS_APY', 'AVAX_APY', 'BADGER_APY', 'BAL_APY', 'BAT_APY', 'BCH_APY', 'BNB_APY', 'BNT_APY', 'BOBA

In [30]:
coin_tier_p1 = coin_tiers[filter2][coin_tier_p1_cols].T
coin_tier_p1.reset_index(inplace = True)
coin_tier_p1.columns = coin_tier_p1.iloc[0]
coin_tier_p1.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p1.columns:
    if col != "Tier":
        new_name.append("Coin_Tier_"+str(col)[0])
coin_tier_p1.columns = new_name
coin_tier_p1

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5
1,1INCH,117269.1238,0.0,0.179374,0.0,6666666.66
2,3CRV,0.0,597.826821,0.0,0.0,0.0
3,AAVE,9715.99047,141.248221,0.0,19659.193429,177720.670178
4,ADA,5437688.80519,0.0,0.0,65511559.0,172989645.6
5,ALCX,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
94,ALICE,0.0,0.0,0.0,20000.527364,0.0
95,POLIS,0.0,0.0,0.0,213558.860739,0.0
96,SRM_LOCKED,0.0,50471.776129,0.0,576735.842576,0.0
97,TRX,0.0,0.0,0.0,0.000003,0.0


In [31]:
coin_tier_p2 = coin_tiers[filter2][coin_tier_p2_cols]
coin_tier_p2.columns = coin_tier_p1_cols
coin_tier_p2 = coin_tier_p2.T
coin_tier_p2.reset_index(inplace = True)
coin_tier_p2.columns = coin_tier_p2.iloc[0]
coin_tier_p2.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p2.columns:
    if col != "Tier":
        new_name.append("APY_Tier_"+str(col)[0])
coin_tier_p2.columns = new_name
coin_tier_p2

,Coin,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
1,1INCH,0.0,0.0,0.0,0.0,0.0
2,3CRV,0.0,0.0,0.0,0.0,0.0
3,AAVE,0.0,0.011224,0.0,0.065771,0.062812
4,ADA,0.0,0.0,0.0,0.0743,0.045
5,ALCX,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
94,ALICE,0.0,0.0,0.0,0.0,0.0
95,POLIS,0.0,0.0,0.0,0.0,0.0
96,SRM_LOCKED,0.0,0.0,0.0,0.0,0.0
97,TRX,0.0,0.0,0.0,0.0,0.0


In [32]:
coin_tiers = coin_tier_p1.merge(coin_tier_p2, on = "Coin", how = "inner")
filter3 = ~coin_tiers["Coin"].isin(stables)
#coin_tiers.columns
coin_tiers

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
0,1INCH,117269.1238,0.0,0.179374,0.0,6666666.66,0.0,0.0,0.0,0.0,0.0
1,3CRV,0.0,597.826821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AAVE,9715.99047,141.248221,0.0,19659.193429,177720.670178,0.0,0.011224,0.0,0.065771,0.062812
3,ADA,5437688.80519,0.0,0.0,65511559.0,172989645.6,0.0,0.0,0.0,0.0743,0.045
4,ALCX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
93,ALICE,0.0,0.0,0.0,20000.527364,0.0,0.0,0.0,0.0,0.0,0.0
94,POLIS,0.0,0.0,0.0,213558.860739,0.0,0.0,0.0,0.0,0.0,0.0
95,SRM_LOCKED,0.0,50471.776129,0.0,576735.842576,0.0,0.0,0.0,0.0,0.0,0.0
96,TRX,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.0,0.0,0.0


## the below code is to generat the pivot_data tab

In [33]:
some_dict = {"Coin":[], "Category":[], "Account": [], "Tier": [], "# of Coins": [], "APY": []}
df1 = pd.DataFrame.from_dict(some_dict)
col_names = ["Coin", "Category", "Account", "Tier", "# of Coins", 
             "APY", "COFA", "USD Value", "USD Value * COFA",
             "USD Value * APY"]
df1

,Coin,Category,Account,Tier,# of Coins,APY


In [34]:
for coin in coin_list:
    if coin not in stables:
        used_cols = ["Account", "Tier", coin, coin+"_APY"]
        df2 = categories.merge(coin_stat_tier_apy[used_cols], on = "Account", how = "right")
        df2["Coin"] = coin
        df2.rename(columns = {coin: "# of Coins", coin+"_APY": "APY"}, inplace = True)
        df1 = pd.concat([df1, df2])
filter1 = df1["# of Coins"] > 10e-6
df1 = df1[filter1]
df1 = df1.merge(coin_price, on = "Coin", how = "left")
df1["USD Value"] = df1["# of Coins"] * df1["Price"]
df1["USD Value * APY"] = df1["USD Value"] * df1["APY"]
df1["Tier"] = df1["Tier"].apply(lambda x:x.split(".")[0])
df1

,Coin,Category,Account,Tier,# of Coins,APY,Price,USD Value,USD Value * APY
0,1INCH,undeployed,Celsius Network System,1,117269.1238,0.0,3.748692,439605.772996,0.0
1,1INCH,Other,Others - Asset,5,6666666.66,NaN,3.748692,24991276.948342,NaN
2,1INCH,Defi,YFL,3,0.179374,NaN,3.748692,0.672416,NaN
3,3CRV,Defi,Deployment - Curve,2,597.826821,NaN,1,597.826821,NaN
4,AAVE,undeployed,Celsius Network System,1,9356.881043,0.0,237.478841,2222061.262659,0.0
...,...,...,...,...,...,...,...,...,...
592,Stable Coins,Exchange,FTX - DeFi,4,5009076.94251,0.0,1,5009076.94251,0.0
593,Stable Coins,Exchange,FTX - Management,4,5450375.60282,0.0,1,5450375.60282,0.0
594,Stable Coins,Exchange,FTX - Kairon,4,222508.4416,0.0,1,222508.4416,0.0
595,Stable Coins,NaN,FTX - Directional Trading 2,nan,134584.13132,0.0,1,134584.13132,0.0


In [36]:
# below 3 code blocks are for analyzing COFA
cats = list(df1["Category"].unique())
accs = list(df1["Account"].unique())
cofa_accts = list(cofa["Account"].unique())
cofa_accts.remove("Default")
cofa_accts

['CEL Treasury',
 'Posted Collateral',
 'FTX - CnC',
 'FTX - CEL',
 'FTX - TEAM Directional',
 'FTX - LONG1',
 'Celsius Borrows Account']

In [37]:
def add_cofa(df1, cofa, cofa_accts):
    default_cofa = cofa[cofa["Account"] == "Default"][["Coin", "COFA"]]
    df2 = df1.merge(default_cofa, on = "Coin", how = "left")
    for col in cofa_accts:
        #print(col)
        cofa_2 = cofa[cofa["Account"] == col]
        if col in cats:
            cofa_2.columns = ["Coin", "Category", "COFA-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Category"], how='left')
        elif col in accs:
            cofa_2.columns = ["Coin", "Account", "COFA-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Account"], how='left')
        else:
            continue
        df2_p1 = df2[df2["COFA-2"].isnull()].drop(columns = ["COFA-2"])
        df2_p2 = df2[df2["COFA-2"].notnull()]
        df2_p2["COFA"] = df2_p2["COFA-2"]
        df2_p2.drop(columns = ["COFA-2"], inplace = True)
        df2 = pd.concat([df2_p1, df2_p2])
        df2.sort_values(by = ["Coin"], inplace = True)
    return df2



In [38]:
df2 = add_cofa(df1, cofa, cofa_accts)
#df2.to_excel("C:/celsius/Liquidity/cofa_2.xlsx", index = False)
df2["USD Value * COFA"] = df2["USD Value"] * df2["COFA"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)
df2

<ipython-input-37-cbe7e601988b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_p2["COFA"] = df2_p2["COFA-2"]
C:\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Coin,Category,Account,Tier,# of Coins,APY,Price,USD Value,USD Value * APY,COFA,USD Value * COFA
298,ETH,Posted Collateral,Celsius Borrows Account,5,592410.1865,0.003000,4108.791368,2434089860.60647,7302269.581819,0.0000,0.0
139,BTC,Posted Collateral,Posted Collateral,5,43405.96,0.000000,54583.87755,2369265605.580198,0.0,0.0000,0.0
130,BTC,Institutional Loans,Loans Out,4,27112.32,0.031400,54583.87755,1479895554.976416,46468720.426259,0.0351,51944333.979672
112,BTC,Exchange,FTX - Jacob,2,26117.836945,0.016096,54583.87755,1425612813.652729,22947376.654961,0.0351,50039009.759211
493,Stable Coins,Institutional Loans,Loans Out,4,1176925372.78,0.108914,1,1176925372.78,128183487.18688,0.1300,153000298.4614
...,...,...,...,...,...,...,...,...,...,...,...
10,AAVE,Exchange,FTX - DeFi,4,0.000062,0.000000,237.478841,0.014776,0.0,0.0384,0.000567
554,VSP,Defi,YieldDesk_Main,5,0.002425,NaN,5.1,0.012366,NaN,NaN,NaN
24,ALICE,Exchange,FTX - CnC,4,20000.527364,NaN,NaN,NaN,NaN,NaN,NaN
395,LRC,Exchange,FTX - CnC,4,900021.23058,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# process the collateral table and insert it into the liquidity tier summary
# for COFA value of collateral, use default first, there is a "Collateral" column in COFA which will override the default
cofa_collateral_p1 = cofa_original[cofa_original["Collateral"].notnull()]
cofa_collateral_p2 = cofa_original[cofa_original["Collateral"].isnull()]
cofa_collateral_p1["Default"] = cofa_collateral_p1["Collateral"]
cofa_collateral = pd.concat([cofa_collateral_p1, cofa_collateral_p2])[["Coin", "Default"]]
cofa_collateral.columns = ["Coin", "COFA"]
collateral = collateral.merge(cofa_collateral, on = "Coin", how = "left")
collateral = collateral.merge(coin_price, on = "Coin", how = "left")
collateral["User Collateral USD Value"] = collateral["User Collateral"] * collateral["Price"]
collateral["Inst Collateral USD Value"] = collateral["Inst Collateral"] * collateral["Price"]


In [40]:
# get the whole apy-template table and insert it into the liquidity tier summary
apy_template = template.drop([0])
apy_template.fillna(value = "N/A", inplace = True)
#some people leave spaces in apy sheet, need to remove them
apy_template.replace([" ", "", "  "], "N/A", inplace = True)

In [41]:
path = "C:/celsius/Liquidity/Liquidity_Tier_Summary.xlsx"
lp = openpyxl.load_workbook(path)
lp.remove(lp['Data'])
lp.remove(lp['Price'])
lp.remove(lp['Pivot_Data'])
lp.remove(lp['APY'])
lp.remove(lp['COFA'])
lp.remove(lp['Collateral'])
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = lp
coin_tiers[filter3].to_excel(writer, sheet_name = 'Data', index = False)
coin_price.to_excel(writer, sheet_name = 'Price', index = False)
df2[col_names].to_excel(writer, sheet_name = 'Pivot_Data', index = False)
apy_template.to_excel(writer, sheet_name = 'APY', index = False)
cofa_original.to_excel(writer, sheet_name = 'COFA', index = False)
collateral.to_excel(writer, sheet_name = 'Collateral', index = False)
writer.close()
lp.close()

## following code will push the data to the shared google sheet 

In [42]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials


In [43]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]

# this is ID for testing
#SPREADSHEET_ID = '1IptNC0hEhwvuyfI4m2kP9-rR-3jDAOhgQaPNkW2I_QQ'
# this is ID for waterfall sheet
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_992941975507-pvjneopi7dhmj2mqq6arpghs6r7q31jj.json', scope)
creds = flow.run_local_server(port=0)
with open('write_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('sheets', 'v4', credentials=creds)

In [44]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_tiers[filter3].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [45]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Price",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Price!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_price.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [46]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="APY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="APY!A1",
    body=dict(
        majorDimension='ROWS',
        values=apy_template.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [47]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="COFA",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="COFA!A1",
    body=dict(
        majorDimension='ROWS',
        values=cofa_original.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [48]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Collateral",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Collateral!A1",
    body=dict(
        majorDimension='ROWS',
        values=collateral.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [49]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Coin_Total_Asset_Liability",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Coin_Total_Asset_Liability!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_asset_liability.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [50]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=df2[col_names].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is to cpoy this freeze waterfall into the archive

In [51]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
'https://www.googleapis.com/auth/drive.file'
]

# this is ID for waterfall live, but we just updated it using freeze data
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('drive_file_token.json'):
        creds = Credentials.from_authorized_user_file('drive_file_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_992941975507-pvjneopi7dhmj2mqq6arpghs6r7q31jj.json', scope)
creds = flow.run_local_server(port=0)
with open('drive_file_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('drive', 'v3', credentials=creds)



In [52]:
# extract the most recent freeze id from the url provided and get the file name using that id
freeze = pd.read_excel("/home/fan7893/Documents/Celsius/Liquidity/freeze_address.xlsx")
freeze.columns = freeze.iloc[0]
freeze.drop([0], inplace = True)
freeze_address = str(freeze.iloc[-1]["Freeze URL"])
#print(freeze_address)
freeze_id = freeze_address.split("/")[5]

#get the file name of most recent freeze sheet, which include the date of the freeze
response = service.files().get(fileId=freeze_id).execute()
freeze_name  = "Portfolio Waterfall - " + response["name"]

In [53]:
# after updating the portfolio waterall live, copied it to the archive folder
archive_folder_id = "13f2xspl16wzdHBRwhgnEkKj80zLL9Rzu"
newfile = {'name': freeze_name,  'parents' : [archive_folder_id]}
response = service.files().copy(fileId=SPREADSHEET_ID, body=newfile).execute()